In [2]:
from adapt_utils import replace_symbols
from my_utils import print_proof_props_graph, get_proof_steps, print_ident_proof, print_proof_linear_steps

from tree_parser import *

#import networkx as nx

#import matplotlib
#import matplotlib.pyplot as plt

import copy

import time

from collections import Counter

import random

In [3]:
%%time

text = file_contents()
database = meta_math_database(text,n=15000)
print("")

included 5555695 tokens from set.mm
proposition: 15000
Wall time: 5min 42s


In [4]:
import re

replace_text_dict = {
    "U_":"∪",
    "|^|":"∩",
    "<.":"⟨",
    ">.":"⟩",
    "+P.":"+P",
    "+Q":"+Q",
    "~~>t": "⇝𝑡",
    "C_":"⊆",
    "C.":"⊊",
    "~<": "≺",
    "~<_": "≼",
    "i^i": "∩",
    "e.":"∈",
    "<->":"↔",
    "->": "→",
    "u.":"∪",
    "-.":"¬",
    "~~":"≈",
    "(/)": "∅",
    "A.": "∀",
    "<_": "≤",
    "=/=": "≠",
    "/\\":"∧",
    "\\/":"∨",
    "E.":"∃",
    "X.": "×",
    "[_":"⦋",
    "]_":"⦌",
    "~P":"𝒫",
    "_V":"V",
    "-/\\": "⊼",
    "\/_": "⊻",
    "_i": "i",
    "ph": "𝜑",
    "ps": "𝜓",
    "ch": "𝜒",
    "th": "𝜃",
    "ta": "𝜏",
    "et": "η",
    "ze": "𝜁",
    "rh": "𝜌",
    "o.": "∘",
    "\\":"∖",
    "/_\\":"△",
    "F/":"Ⅎ",
    "RR":"ℝ",
    "CC":"ℂ",
    "e/":"∉",
    "E*":"∃*",
    "≺_":"≼",
    "+Q":"+Q",
    "x.":"·",
    "∨_":"⊻",
    "|→":"↦",
    #"-":"−",
    "-→":"⟶",
    "NN":"ℕ",
    "ZZ":"ℤ",
    "QQ":"ℚ",
    "-u":"-",
    '"':'“'
}

re_str = "|".join(map(re.escape, replace_text_dict.keys()))

def replace_symbols(text):
    return re.sub(re_str, lambda a: replace_text_dict[a.group()], text)

In [5]:
theorems = [
    "ssequn1",
    "ssdif0",
    #"opoeALTV",
    #"epee",
    "dfss2",
    "sqrt2irr",
    "disj3",
    "difin",
    "eldif",
    "dfss4",
    "nn0ssz",
    "indi"
]

In [6]:
def print_proof_linear_steps(p, database):
    prop = database.propositions[p]
    for s in prop.entails_proof_steps:
        print("[{}]".format(s.prop.label), "|-", replace_symbols(s.tree.eval(database, prop)))

In [7]:
def print_linear_steps(prop):
    #prop = "ssequn1"

    print("\t" + replace_symbols(database.propositions[prop].statement_str()))

    print_proof_linear_steps(prop, database)
    print()

In [8]:
for t in theorems:
    print_linear_steps(t)

	|- ( A ⊆ B ↔ ( A ∪ B ) = B )
[bicom] |- ( ( x ∈ B ↔ ( x ∈ A ∨ x ∈ B ) ) ↔ ( ( x ∈ A ∨ x ∈ B ) ↔ x ∈ B ) )
[pm4.72] |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ B ↔ ( x ∈ A ∨ x ∈ B ) ) )
[elun] |- ( x ∈ ( A ∪ B ) ↔ ( x ∈ A ∨ x ∈ B ) )
[bibi1i] |- ( ( x ∈ ( A ∪ B ) ↔ x ∈ B ) ↔ ( ( x ∈ A ∨ x ∈ B ) ↔ x ∈ B ) )
[3bitr4i] |- ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
[albii] |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
[dfss2] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
[dfcleq] |- ( ( A ∪ B ) = B ↔ ∀ x ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
[3bitr4i] |- ( A ⊆ B ↔ ( A ∪ B ) = B )

	|- ( A ⊆ B ↔ ( A ∖ B ) = ∅ )
[iman] |- ( ( x ∈ A → x ∈ B ) ↔ ¬ ( x ∈ A ∧ ¬ x ∈ B ) )
[eldif] |- ( x ∈ ( A ∖ B ) ↔ ( x ∈ A ∧ ¬ x ∈ B ) )
[xchbinxr] |- ( ( x ∈ A → x ∈ B ) ↔ ¬ x ∈ ( A ∖ B ) )
[albii] |- ( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ¬ x ∈ ( A ∖ B ) )
[dfss2] |- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
[eq0] |- ( ( A ∖ B ) = ∅ ↔ ∀ x ¬ x ∈ ( A ∖ B ) )
[3bitr4i] |- ( A ⊆ B ↔ ( A ∖ B ) = ∅ )

	|- ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
[dfs

In [61]:
theorems_answers = dict()

for t in theorems:
    
    prop_statement = replace_symbols(database.propositions[t].statement_str())[3:]
    
    print(prop_statement)

    prop = database.propositions[t]
    
    prop_steps = []
    
    for s in prop.entails_proof_steps:
        prop_steps.append(replace_symbols(s.tree.eval(database, prop)))
        #print("[{}]".format(s.prop.label), "|-", replace_symbols(s.tree.eval(database, prop)))
        
    prop_steps = sorted(prop_steps, key=lambda a: len(a))
    
    theorems_answers[t] = {
        "label":t,
        "statement":prop_statement,
        "sample_steps":[]
    }
    
    for step in prop_steps:
        if step ==  prop_statement:
            continue
        theorems_answers[t]["sample_steps"].append(step)
        print(step)

    print()

( A ⊆ B ↔ ( A ∪ B ) = B )
( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
( x ∈ ( A ∪ B ) ↔ ( x ∈ A ∨ x ∈ B ) )
( ( x ∈ A → x ∈ B ) ↔ ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
( ( A ∪ B ) = B ↔ ∀ x ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
( ( x ∈ A → x ∈ B ) ↔ ( x ∈ B ↔ ( x ∈ A ∨ x ∈ B ) ) )
( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
( ( x ∈ ( A ∪ B ) ↔ x ∈ B ) ↔ ( ( x ∈ A ∨ x ∈ B ) ↔ x ∈ B ) )
( ( x ∈ B ↔ ( x ∈ A ∨ x ∈ B ) ) ↔ ( ( x ∈ A ∨ x ∈ B ) ↔ x ∈ B ) )

( A ⊆ B ↔ ( A ∖ B ) = ∅ )
( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
( x ∈ ( A ∖ B ) ↔ ( x ∈ A ∧ ¬ x ∈ B ) )
( ( x ∈ A → x ∈ B ) ↔ ¬ x ∈ ( A ∖ B ) )
( ( A ∖ B ) = ∅ ↔ ∀ x ¬ x ∈ ( A ∖ B ) )
( ( x ∈ A → x ∈ B ) ↔ ¬ ( x ∈ A ∧ ¬ x ∈ B ) )
( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ¬ x ∈ ( A ∖ B ) )

( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
( A ⊆ B ↔ A = ( A ∩ B ) )
( A ∩ B ) = { x | ( x ∈ A ∧ x ∈ B ) }
( A ⊆ B ↔ ∀ x ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
( A = ( A ∩ B ) ↔ A = { x | ( x ∈ A ∧ x ∈ B ) } )
( ( x ∈ A → x ∈ B ) ↔ ( x ∈ A ↔ ( x ∈ A ∧ x ∈ B ) ) )
( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ A 

In [62]:
theorems_answers

{'ssequn1': {'label': 'ssequn1',
  'statement': '( A ⊆ B ↔ ( A ∪ B ) = B )',
  'sample_steps': ['( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )',
   '( x ∈ ( A ∪ B ) ↔ ( x ∈ A ∨ x ∈ B ) )',
   '( ( x ∈ A → x ∈ B ) ↔ ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )',
   '( ( A ∪ B ) = B ↔ ∀ x ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )',
   '( ( x ∈ A → x ∈ B ) ↔ ( x ∈ B ↔ ( x ∈ A ∨ x ∈ B ) ) )',
   '( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )',
   '( ( x ∈ ( A ∪ B ) ↔ x ∈ B ) ↔ ( ( x ∈ A ∨ x ∈ B ) ↔ x ∈ B ) )',
   '( ( x ∈ B ↔ ( x ∈ A ∨ x ∈ B ) ) ↔ ( ( x ∈ A ∨ x ∈ B ) ↔ x ∈ B ) )']},
 'ssdif0': {'label': 'ssdif0',
  'statement': '( A ⊆ B ↔ ( A ∖ B ) = ∅ )',
  'sample_steps': ['( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )',
   '( x ∈ ( A ∖ B ) ↔ ( x ∈ A ∧ ¬ x ∈ B ) )',
   '( ( x ∈ A → x ∈ B ) ↔ ¬ x ∈ ( A ∖ B ) )',
   '( ( A ∖ B ) = ∅ ↔ ∀ x ¬ x ∈ ( A ∖ B ) )',
   '( ( x ∈ A → x ∈ B ) ↔ ¬ ( x ∈ A ∧ ¬ x ∈ B ) )',
   '( ∀ x ( x ∈ A → x ∈ B ) ↔ ∀ x ¬ x ∈ ( A ∖ B ) )']},
 'dfss2': {'label': 'dfss2',
  'statement': '( A ⊆ B ↔ ∀ x ( x ∈ A →

In [63]:
### Sample answers

include_symbols = [
    "⊆",
    "∈",
    "∩",
    #"∧",
    "∪",
    #"∨",
    #"∅",
    "△", #symmetric difference,
]

exclude_symbols = [
    "Ⅎ",
    "V",
    "⦋",
    "⦌",
    "𝒫",
    "<N",
    "o",
    "Fin",
    "suc",
    "U.",
    "<Q",
    "card",
    "`",
    "Tr",
    "Fr",
    "On",
    "≺",
    "≼",
    "Inacc",
    "Rel",
    "<R",
    "0R",
    "Tarski",
    "~Q",
    "Fn",
    "Ord",
    "≈",
    "1P",
    "1Q",
    "+Q",
    "E!",
    "WUni",
    "ℝ*",
    "⟩",
    "x.",
    "<P",
    "~R",
    "P.",
    "-1R",
    "<ℝ",
    "R.",
    ".Q",
    "We",
    "+N",
    "Lim",
    "Q.",
    "Or",
    "_E",
    "N.",
    "T.",
    "GCH",
    "ale𝜑",
    "CHOICE",
    "Disj_",
    "Fun",
    "Univ",
    "ran",
    "supp",
    "finSupp",
    "+P",
    "∨_",
    "[.",
    "A⊆",
    "∩_",
    "^pm",
    "^m",
    "E*",
    "X_",
    "_I",
    "|→",
    "-→",
    "inf",
    "sup",
    "if",
    "∃*",
    "+e",
    "[,]",
    "(,)",
    "“",
    "−→",
    "*e",
    ";",
    "−e",
    "−u",
    "-e"
]

expression_lists = []
labels_list = []
for p in database.propositions_list:
    
    for s in p.entails_proof_steps:
        try:
            step_expr = replace_symbols(s.tree.eval(database, prop))
            
            #Check if some excluding symbol is present
            skip_step = False 
            for exc in exclude_symbols:
                if exc in step_expr:
                    skip_step = True
                    break    
            if skip_step:
                continue
            
            #Check if some including symbol is present
            for inc in include_symbols:
                if inc in step_expr and step_expr not in expression_lists:
                    #expression_lists.append((s.prop.label, step_expr))
                    expression_lists.append(step_expr)
                    labels_list.append(s.prop.label)
                    break
        except:
            pass



In [64]:
terms = set()

for e in expression_lists:
    for t in e.split(" "):
        terms.add(t)

len(terms)

56

In [65]:
terms

{'(',
 ')',
 '+',
 ',',
 '-',
 '/',
 '0',
 '1',
 '10',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '<',
 '=',
 'A',
 'B',
 'C',
 '[,)',
 'i',
 'x',
 '{',
 '|',
 '}',
 '¬',
 '·',
 '×',
 'ℂ',
 'ℕ',
 'ℕ0',
 'ℚ',
 'ℝ',
 'ℝ+',
 'ℤ',
 '→',
 '↔',
 '∀',
 '∃',
 '∅',
 '∈',
 '∉',
 '∖',
 '∘',
 '∧',
 '∨',
 '∩',
 '∪',
 '≠',
 '≤',
 '⊆',
 '⊊',
 '△'}

In [66]:
def show_filtered_expressions(term, limit=-1):
    
    i = 0
    for l, e in zip(labels_list, expression_lists):
        if term in e:
            print(l,e)
            i += 1
            if i > 0 and i > limit:
                break
                
show_filtered_expressions('×', limit=10)

mtbir ¬ ∅ ∈ ( A × B )
sylbi ( C ∈ ( A × B ) → ¬ ∅ ∈ C )
eqtr4i ( A × ( B ∪ C ) ) = ( ( A × B ) ∪ ( A × C ) )
eqtr4i ( ( A ∪ B ) × C ) = ( ( A × C ) ∪ ( B × C ) )
eqriv ( C × ∪ x ∈ A B ) = ∪ x ∈ A ( C × B )
eqriv ( ∪ x ∈ A B × C ) = ∪ x ∈ A ( B × C )
xpiundir ( ∪ x ∈ A { x } × B ) = ∪ x ∈ A ( { x } × B )
xpeq1i ( ∪ x ∈ A { x } × B ) = ( A × B )
eqtr3i ∪ x ∈ A ( { x } × B ) = ( A × B )
xpss12 ( ( A ⊆ B ∧ C ⊆ C ) → ( A × C ) ⊆ ( B × C ) )
mpan2 ( A ⊆ B → ( A × C ) ⊆ ( B × C ) )


In [67]:
#Generate questionaires

len_margin = 3

questionaire = []

for t in list(theorems_answers.values()):
    for i in range(3):
        correct_option = t['sample_steps'][i]
        alternatives = [correct_option]
        #Generate alternatives
        random.shuffle(expression_lists)
        for e in expression_lists:
            correct_len = len(correct_option)
            if correct_len - len_margin < len(e) < correct_len + len_margin and e not in t['sample_steps']:
                alternatives.append(e)
                if len(alternatives) >= 5:
                    break 
                    
        questionaire.append({
            "statement": t["statement"],
            "alternatives": alternatives
        })

In [68]:
questionaire

[{'statement': '( A ⊆ B ↔ ( A ∪ B ) = B )',
  'alternatives': ['( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )',
   '( A ⊆ C ↔ ∀ x ( x ∈ A → x ∈ C ) )',
   '( ∩ A ⊆ x → ( x ⊆ A → ∩ A ⊆ A ) )',
   '( A ⊆ B → ( A × C ) ⊆ ( B × C ) )',
   '( ( B ∈ ℝ ∧ B < 0 ) → - B ∈ ℝ )']},
 {'statement': '( A ⊆ B ↔ ( A ∪ B ) = B )',
  'alternatives': ['( x ∈ ( A ∪ B ) ↔ ( x ∈ A ∨ x ∈ B ) )',
   '( ∃ x ∈ A ¬ x = ∅ ↔ ¬ ∀ x ∈ A x = ∅ )',
   '( x ⊆ { ∅ } → ( ¬ x = ∅ → x = { ∅ } ) )',
   '( A ∈ ℂ → ( ( A / 2 ) = 0 ↔ A = 0 ) )',
   '( A ∈ ℝ → - ( A - 1 ) = ( - A + 1 ) )']},
 {'statement': '( A ⊆ B ↔ ( A ∪ B ) = B )',
  'alternatives': ['( ( x ∈ A → x ∈ B ) ↔ ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )',
   '( ( A ∈ ℂ ∧ B ∈ ℂ ∧ B ≠ 0 ) → - ( B / B ) = - 1 )',
   '( B ∈ { A , B , C } ↔ ( B = A ∨ B = B ∨ B = C ) )',
   '( ( A ∈ ℝ ∧ ( B ∈ ℝ ∧ 0 < B ) ∧ C ∈ ℝ ) → C ∈ ℝ )',
   '( A ∈ { A , B , C } ↔ ( A = A ∨ A = B ∨ A = C ) )']},
 {'statement': '( A ⊆ B ↔ ( A ∖ B ) = ∅ )',
  'alternatives': ['( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )',
   '( A ∖ B ) 

In [69]:
#Plot Questionaire

for i, q in enumerate(questionaire):
    teorema = q["statement"]
    print(f"{i+1}) Qual das expressões abaixo corresponde a um passo da prova do teorema {teorema}?")
    alternativas = q["alternatives"]
    #random.shuffle(alternativas)
    for j, a in zip(["a", "b", "c", "d", "e"], alternativas):
        print(f"\t{j}. "+ a)
        
    print("\n\n")

1) Qual das expressões abaixo corresponde a um passo da prova do teorema ( A ⊆ B ↔ ( A ∪ B ) = B )?
	a. ( A ⊆ B ↔ ∀ x ( x ∈ A → x ∈ B ) )
	b. ( A ⊆ C ↔ ∀ x ( x ∈ A → x ∈ C ) )
	c. ( ∩ A ⊆ x → ( x ⊆ A → ∩ A ⊆ A ) )
	d. ( A ⊆ B → ( A × C ) ⊆ ( B × C ) )
	e. ( ( B ∈ ℝ ∧ B < 0 ) → - B ∈ ℝ )



2) Qual das expressões abaixo corresponde a um passo da prova do teorema ( A ⊆ B ↔ ( A ∪ B ) = B )?
	a. ( x ∈ ( A ∪ B ) ↔ ( x ∈ A ∨ x ∈ B ) )
	b. ( ∃ x ∈ A ¬ x = ∅ ↔ ¬ ∀ x ∈ A x = ∅ )
	c. ( x ⊆ { ∅ } → ( ¬ x = ∅ → x = { ∅ } ) )
	d. ( A ∈ ℂ → ( ( A / 2 ) = 0 ↔ A = 0 ) )
	e. ( A ∈ ℝ → - ( A - 1 ) = ( - A + 1 ) )



3) Qual das expressões abaixo corresponde a um passo da prova do teorema ( A ⊆ B ↔ ( A ∪ B ) = B )?
	a. ( ( x ∈ A → x ∈ B ) ↔ ( x ∈ ( A ∪ B ) ↔ x ∈ B ) )
	b. ( ( A ∈ ℂ ∧ B ∈ ℂ ∧ B ≠ 0 ) → - ( B / B ) = - 1 )
	c. ( B ∈ { A , B , C } ↔ ( B = A ∨ B = B ∨ B = C ) )
	d. ( ( A ∈ ℝ ∧ ( B ∈ ℝ ∧ 0 < B ) ∧ C ∈ ℝ ) → C ∈ ℝ )
	e. ( A ∈ { A , B , C } ↔ ( A = A ∨ A = B ∨ A = C ) )



4) Qual das expressõe

In [70]:
verificar comentarios edson e matheus e continuar resolvendo isso

SyntaxError: invalid syntax (<ipython-input-70-53a38252ec78>, line 1)